<a href="https://colab.research.google.com/github/uditnegi16/imagedenoiser/blob/main/imagedenolise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import tensorflow as tf
import os
# List all image paths
TRAIN_NOISY_DIR = '/content/drive/MyDrive/imagedenoise/imagedenoise/BSDS300/images/train/noisy'
TRAIN_CLEAN_DIR= '/content/drive/MyDrive/imagedenoise/imagedenoise/BSDS300/images/train/clean'

In [ ]:
# Get the dimensions of the noisy and clean images
for images in train_noisy_dataset.skip(40).take(1):
    noisy_image = images[0]
    print("Noisy image dimensions:", noisy_image.shape)

for images in train_clean_dataset.take(1):
    clean_image = images[0]
    print("Clean image dimensions:", clean_image.shape)

In [ ]:
rescale_layer = tf.keras.layers.Rescaling(scale=1./255)
# Combine the noisy and clean datasets
train_dataset = tf.data.Dataset.zip((train_noisy_dataset, train_clean_dataset))
# Apply the normalization function to the combined dataset
# # Apply the rescaling layer to the combined dataset
train_dataset_scaled = train_dataset.map(lambda x, y: (rescale_layer(x), rescale_layer(y)))
# train_dataset_scaled = train_dataset.map(lambda noisy, clean: (rescale_layer(noisy), rescale_layer(clean)))/

In [ ]:
train_dataset_scaled = train_dataset.map(lambda x, y: (tf.image.per_image_standardization(x), tf.image.per_image_standardization(y)))

In [ ]:
# Print image values before rescaling
for noisy, clean in train_dataset.take(1):
    print("Before rescaling:")
    print("Noisy image values:", noisy[0, 0, :5])  # print the first 5 pixel values of the first row of the noisy image
    print("Clean image values:", clean[0, 0, :5])  # print the first 5 pixel values of the first row of the clean image

# Apply rescaling
train_dataset_scaled = train_dataset.map(lambda x, y: (rescale_layer(x), rescale_layer(y)))

# Print image values after rescaling
for noisy, clean in train_dataset_scaled.take(1):
    print("After rescaling:")
    print("Noisy image values:", noisy[0, 0, :5])  # print the first 5 pixel values of the first row of the rescaled noisy image
    print("Clean image values:", clean[0, 0, :5])  # print the first 5 pixel values of the first row of the rescaled clean image

In [ ]:
# Use a for loop to print the shapes of the clean and noisy images
for clean_image, noisy_image in train_dataset_scaled.take(1):
    print(f"Clean image shape: {clean_image.shape}, Noisy image shape: {noisy_image.shape}")

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers

# PSNR metric
def psnr_loss(y_true, y_pred):
    max_pixel_value = 255.0
    return tf.image.psnr(y_true, y_pred, max_pixel_value)

# Define the input shape
input_shape = (448, 384, 3)

model = tf.keras.models.Sequential([
    # Encoder
    layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=input_shape),
    layers.MaxPooling2D((2, 2), padding='same'),

    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2), padding='same'),

    layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    layers.MaxPooling2D((2, 2), padding='same'),

    # Decoder
    layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),

    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.UpSampling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.UpSampling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.UpSampling2D((2, 2)),

    # Output Layer
    layers.Conv2D(3, (3, 3), activation='linear', padding='same')
])

# Compilation
model.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate=1e-4, weight_decay=1e-5),
    loss='mean_squared_error',
    metrics=[psnr_loss]
)

model.summary()

In [ ]:
model.fit(
   train_dataset_scaled,
    epochs=30,
    verbose=1
    # validation_data=validation_dataset_scaled
)

In [ ]:

# Step 2: Specify the path where you want to save the model
model_save_path = '/content/drive/MyDrive/imagedenoise/denoising_model.keras'

# Step 3: Save the trained model
model.save(model_save_path)

print(f"Model saved to {model_save_path}")

Model saved to /content/drive/MyDrive/imagedenoise/denoising_model.keras


In [ ]:
TEST_NOISY_DIR = '/content/drive/MyDrive/imagedenoise/imagedenoise/BSDS300/images/test/noisy'
TEST_CLEAN_DIR = '/content/drive/MyDrive/imagedenoise/imagedenoise/BSDS300/images/test/clean'

# Create the test noisy and clean datasets
test_noisy_dataset = tf.keras.utils.image_dataset_from_directory(
    TEST_NOISY_DIR,
    image_size=(448, 384),  # Resize images to 448x384
    batch_size=3,
    label_mode=None
)

test_clean_dataset = tf.keras.utils.image_dataset_from_directory(
    TEST_CLEAN_DIR,
    image_size=(448, 384),  # Resize images to 448x384
    batch_size=3,
    label_mode=None
)

In [ ]:
rescale_layer = tf.keras.layers.Rescaling(scale=1./255)
# Combine the noisy and clean datasets
test_dataset = tf.data.Dataset.zip((test_noisy_dataset, test_clean_dataset))

# Apply the rescaling layer to the combined dataset
test_dataset_scaled = test_dataset.map(lambda x, y: (tf.image.per_image_standardization(x), tf.image.per_image_standardization(y)))

In [ ]:
# Evaluate the model on the test dataset
test_loss, test_psnr = model.evaluate(test_dataset_scaled, verbose=1)

print("Test loss:", test_loss)
print("Test PSNR:", test_psnr)

# Generate predictions on the test dataset
test_predictions = model.predict(test_dataset_scaled)
# 3s - 306ms/step - loss: 0.1958 - psnr_loss: 61.0361
# Test loss: 0.19577130675315857
# Test PSNR: 61.03607940673828
# # 9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 135ms

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from skimage.io import imsave
import matplotlib.pyplot as plt

# Load the trained CNN denoising model
# model = keras.models.load_model('/content/drive/MyDrive/imagedenoise/denoising_model.h5')

# Load the PNG image from Google Drive
image_path = '/content/drive/MyDrive/imagedenoise/15004.jpg'
image_string = tf.io.read_file(image_path)
img_noisy = tf.image.decode_png(image_string, channels=3)

# Resize the image to (448, 384) as in the training dataset
img_noisy = tf.image.resize(img_noisy, (448, 384))

# Apply the same rescaling used during training
img_noisy = tf.cast(img_noisy, tf.float32) / 255.0  # Normalize to [0, 1]

# Add a batch dimension (necessary for model prediction)
img_noisy = tf.expand_dims(img_noisy, axis=0)

# Pass the noisy image to the denoising model
denoised_img = model.predict(img_noisy)

# Convert the denoised image back to the original range [0, 255]
denoised_img = denoised_img[0] * 255.0  # Remove batch dimension and rescale
denoised_img = tf.clip_by_value(denoised_img, 0, 255)  # Clip values to valid range
denoised_img = denoised_img.numpy().astype(np.uint8)  # Convert to uint8 for saving and displaying

# Save the denoised image
imsave('denoised_image.png', denoised_img)

# Display the denoised image
plt.imshow(denoised_img)
plt.axis('off')  # Hide the axes for cleaner display
plt.show()


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from skimage.io import imsave
import matplotlib.pyplot as plt

# Load the trained CNN denoising model/
# model = keras.models.load_model('/content/drive/MyDrive/imagedenoise/denoising_model.h5')

# Load the PNG image from Google Drive
image_path = '/content/drive/MyDrive/imagedenoise/gaussimage.png'
image_string = tf.io.read_file(image_path)
img_noisy = tf.image.decode_png(image_string, channels=3)

# Step 1: Display the loaded image before any preprocessing
plt.imshow(img_noisy.numpy())
plt.title("Original Loaded Image")
plt.show()

# Resize the image to (448, 384) as in the training dataset
img_noisy = tf.image.resize(img_noisy, (448, 384))

# Step 2: Display the resized image
plt.imshow(img_noisy.numpy().astype(np.uint8))
plt.title("Resized Image (448x384)")
plt.show()

# Apply the same rescaling used during training
img_noisy = tf.cast(img_noisy, tf.float32) / 255.0  # Normalize to [0, 1]

# Step 3: Check if normalization is working as expected
print("Noisy image max value (after normalization):", tf.reduce_max(img_noisy).numpy())
print("Noisy image min value (after normalization):", tf.reduce_min(img_noisy).numpy())

# Add a batch dimension (necessary for model prediction)
img_noisy = tf.expand_dims(img_noisy, axis=0)

# Pass the noisy image to the denoising model
denoised_img = model.predict(img_noisy)

# Convert the denoised image back to the original range [0, 255]
denoised_img = denoised_img[0] * 255.0  # Remove batch dimension and rescale
denoised_img = tf.clip_by_value(denoised_img, 0, 255)  # Clip values to valid range
denoised_img = denoised_img.numpy().astype(np.uint8)  # Convert to uint8 for saving and displaying

# Step 4: Display the denoised image after model prediction
plt.imshow(denoised_img)
plt.title("Denoised Image")
plt.axis('off')
plt.show()

# Save the denoised image
imsave('denoised_image.png', denoised_img)
